In [ ]:
import importlib
import subprocess
import sys
import gc

def check_and_install_package(package_name, version=None):
    try:
        importlib.import_module(package_name)
        print(f"\n{package_name} is already installed.")
    except ImportError:
        print(f"\n{package_name} is NOT installed. Installing now...")
        if version:
            subprocess.check_call([sys.executable, "-m", "pip", "install", f"{package_name}=={version}"])
        else:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"{package_name} installation completed.")

# List of packages to check along with specific versions if necessary
packages = [
    {"name": "tqdm", "version": None},
    {"name": "pyspark", "version": "3.5.2"},
    {"name": "gdown", "version": None},
    {"name": "numpy", "version": "1.23.5"}
]

# Checking and installing the packages
for package in packages:
    check_and_install_package(package["name"], package["version"])


tqdm is already installed.

pyspark is NOT installed. Installing now...
pyspark installation completed.

gdown is already installed.

numpy is already installed.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("LinearRegressor") \
    .config("spark.driver.memory", "40g") \
    .config("spark.executor.memory", "40g") \
    .config("spark.driver.maxResultSize", "5g") \
    .config("spark.executor.memoryOverhead", "8g") \
    .config("spark.executor.cores", "5") \
    .config("spark.kryoserializer.buffer.max", "2047m") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "400") \
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:InitiatingHeapOccupancyPercent=35 -XX:ConcGCThreads=4 -XX:ParallelGCThreads=4") \
    .getOrCreate()

# Verifying Spark session creation
print(f"Spark session started with version: {spark.version}")


Spark session started with version: 3.5.2


In [ ]:
# loading the df

!cp '/content/drive/MyDrive/Big Data Analytics - Project/Datasets/Processed_DF.parquet' /content/

output_path = '/content/Processed_DF.parquet'
df = spark.read.parquet(output_path)
print("The Processed DataFrame has been loaded successfully.")


The Processed DataFrame has been loaded successfully.


In [ ]:
df = df.repartition(100)  # Repartitioning into 100 partitions for parallelism

In [ ]:
# Printing the shape of the DataFrame
total_rows = df.count()
total_columns = len(df.columns)

print(f"The shape of the loaded DataFrame is: ({total_rows}, {total_columns})")

The shape of the loaded DataFrame is: (3000040, 42)


In [ ]:
from functools import reduce
from pyspark.sql.functions import col

# Checking for rows with at least one null value in any column
rows_with_null = df.filter(
    reduce(lambda x, y: x | y, (col(c).isNull() for c in df.columns))).count()

print(f"Number of rows with at least one null value: {rows_with_null}")

Number of rows with at least one null value: 0


### **Handling Categorical Coloumns**

In [ ]:
df=df.drop('description','major_options','mileage')
# Keeping the columns ['exterior_color','dealer_zip','interior_color']

In [ ]:
# Counting unique values in 'exterior_color' and 'interior_color' columns
exterior_colors_count = df.select('exterior_color').distinct().count()
interior_colors_count = df.select('interior_color').distinct().count()

print(f"Unique exterior colors: {exterior_colors_count}")
print(f"Unique interior colors: {interior_colors_count}")

Unique exterior colors: 23036
Unique interior colors: 38528


In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType

# Listing top colors for both exterior and interior
colors = ['White', 'Black', 'Gray', 'Silver', 'Red', 'Blue', 'Brown', 'Green', 'Beige', 'Orange', 'Gold', 'Yellow', 'Purple']

# Creating a UDF to find colors in the color columns
@F.udf(returnType=ArrayType(StringType()))
def find_colors(color_string):
    if color_string is None or color_string.strip() == "":
        return ["Other"]  # Handle empty or null values
    found_colors = [c for c in colors if c.lower() in color_string.lower()]
    return found_colors if found_colors else ["Other"]  # Label non-matching colors as "Other"

# Applying the UDF to both the exterior and interior color columns
df = df.withColumn("exterior_color_array", find_colors("exterior_color"))
df = df.withColumn("interior_color_array", find_colors("interior_color"))

# Creating a column with the count of colors found for both exterior and interior
df = df.withColumn("exterior_color_count", F.size("exterior_color_array"))
df = df.withColumn("interior_color_count", F.size("interior_color_array"))

# Joining the color arrays into string columns
df = df.withColumn("exterior_color", F.array_join("exterior_color_array", ", "))
df = df.withColumn("interior_color", F.array_join("interior_color_array", ", "))

# Labeling mixed colors for both exterior and interior colors
df = df.withColumn(
    "exterior_color",
    F.when(F.col("exterior_color_count") > 1, "Mixed Colors")
     .otherwise(F.col("exterior_color")))

df = df.withColumn(
    "interior_color",
    F.when(F.col("interior_color_count") > 1, "Mixed Colors")
     .otherwise(F.col("interior_color")))

# Dropping temporary columns
df = df.drop("exterior_color_array", "exterior_color_count", "interior_color_array", "interior_color_count")


In [ ]:
# Counting the occurrences of each exterior and interior color and calculating percentages
exterior_color_counts = df.groupBy("exterior_color").count().withColumn(
    "percentage", F.round((F.col("count") / df.count()) * 100, 2))

interior_color_counts = df.groupBy("interior_color").count().withColumn(
    "percentage", F.round((F.col("count") / df.count()) * 100, 2))


# Showing the results
print("Exterior Color Distribution:")
exterior_color_counts.orderBy(F.desc("count")).show(truncate=False)

Exterior Color Distribution:
+--------------+------+----------+
|exterior_color|count |percentage|
+--------------+------+----------+
|White         |675979|22.53     |
|Black         |580148|19.34     |
|Other         |543638|18.12     |
|Silver        |384540|12.82     |
|Blue          |253263|8.44      |
|Red           |242331|8.08      |
|Gray          |231172|7.71      |
|Green         |23026 |0.77      |
|Mixed Colors  |19728 |0.66      |
|Brown         |12905 |0.43      |
|Orange        |11638 |0.39      |
|Gold          |10544 |0.35      |
|Beige         |5065  |0.17      |
|Yellow        |4855  |0.16      |
|Purple        |1208  |0.04      |
+--------------+------+----------+



In [ ]:
print("Interior Color Distribution:")
interior_color_counts.orderBy(F.desc("count")).show(truncate=False)

Interior Color Distribution:
+--------------+-------+----------+
|interior_color|count  |percentage|
+--------------+-------+----------+
|Black         |1624033|54.13     |
|Other         |577578 |19.25     |
|Gray          |383966 |12.8      |
|Mixed Colors  |171212 |5.71      |
|White         |91545  |3.05      |
|Brown         |65943  |2.2       |
|Red           |34117  |1.14      |
|Silver        |24124  |0.8       |
|Blue          |22828  |0.76      |
|Green         |2048   |0.07      |
|Gold          |1193   |0.04      |
|Orange        |1133   |0.04      |
|Yellow        |134    |0.0       |
|Purple        |121    |0.0       |
|Beige         |65     |0.0       |
+--------------+-------+----------+



In [ ]:
print(f"Final processed DataFrame used for the model has {df.count()} rows and {len(df.columns)} columns.")

Final processed DataFrame used for the model has 3000040 rows and 39 columns.


In [ ]:
# Calculating the average price
avg_price = df.agg({"price": "avg"}).collect()[0][0]
print(f"Average price of a car: {round(avg_price)}")


Average price of a car: 29933


In [ ]:
import pandas as pd
from IPython.display import display
import pyspark.sql.functions as F

# Converting the Spark DataFrame to a Pandas DataFrame and displaying the first 5 rows
pd.set_option('display.max_columns', None)
pandas_df = df.orderBy(F.rand()).limit(5).toPandas()
display(pandas_df)


,fuel_type,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,engine_type,exterior_color,franchise_dealer,fuel_tank_volume,height,highway_fuel_economy,horsepower,interior_color,is_new,latitude,length,listed_date,listing_color,longitude,make_name,maximum_seating,model_name,price,savings_amount,seller_rating,sp_name,torque,transmission,transmission_display,wheel_system_display,wheelbase,width,year,combined_fuel_economy,legroom,log_mileage,major_options_count
0,Gasoline,SUV / Crossover,Buford,13.0,47,30518,4200.0,I6,Gray,False,22.0,71.9,19.0,275.0,Gray,False,34.137199,191.6,2020-07-25,GRAY,-83.973198,GMC,5.0,Envoy,3495.0,0,3.730769,Global Imports Auto Sales,275.0,A,4-Speed Automatic,Four-Wheel Drive,113.0,74.7,2003,16.0,81.7,12.25,3
1,Flex Fuel Vehicle,SUV / Crossover,Las Vegas,13.0,26,89149,5700.0,V8,Gold,True,26.4,77.0,17.0,381.0,Gray,False,36.276699,205.1,2020-08-16,GOLD,-115.272003,Toyota,8.0,Sequoia,38999.0,806,3.625000,Prestige Chrysler Jeep Dodge,401.0,A,Automatic,Four-Wheel Drive,122.0,79.9,2016,15.0,83.4,10.91,8
2,Gasoline,SUV / Crossover,Madison,16.0,15,53713,3600.0,V6,Gray,True,21.1,66.6,24.0,283.0,Black,False,43.032398,192.4,2020-08-26,GRAY,-89.406097,Dodge,7.0,Journey,22976.0,734,4.428571,Bergstrom Chevrolet of Madison,260.0,A,6-Speed Automatic,All-Wheel Drive,113.8,83.7,2018,20.0,76.9,9.86,11
3,Gasoline,Sedan,Brick,26.0,273,08724,2500.0,I4,Blue,True,16.4,57.1,35.0,187.0,Black,True,40.084499,192.7,2019-12-11,BLUE,-74.095596,Mazda,5.0,MAZDA6,27520.0,0,4.166667,All American Ford & Mazda of Point Pleasant,186.0,A,Automatic,Front-Wheel Drive,111.4,82.3,2020,30.5,80.9,2.08,5
4,Gasoline,SUV / Crossover,Taylor,21.0,34,18517,2400.0,I4,Blue,False,15.9,66.2,28.0,180.0,Black,False,41.383801,182.0,2020-08-07,BLUE,-75.709396,Jeep,5.0,Cherokee,19998.0,2503,5.000000,Gaughan Auto Store,239.0,A,9-Speed Automatic,Four-Wheel Drive,106.3,73.2,2018,24.5,81.4,11.19,3


# **Linear Regression**

### **20% of the dataset**

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
import time
import warnings
from tqdm import tqdm
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder
from pyspark.ml import Pipeline
import pyspark.sql.functions as F

# Ignore warnings
warnings.filterwarnings('ignore')

# Starting to track overall runtime
start_time = time.time()

with tqdm(total=6, desc="Processing and Training") as pbar:

    df_sample = df.sample(fraction=0.2, seed=42)  # Randomly sampling 20% of the data
    pbar.update(1)

    # Handling categorical columns
    cat_columns = [field for (field, dtype) in df_sample.dtypes.items() if dtype == "string"]
    stages = []
    for col_name in cat_columns:
        indexer = StringIndexer(inputCol=col_name, outputCol=f"{col_name}_indexed", handleInvalid="keep")
        encoder = OneHotEncoder(inputCols=[f"{col_name}_indexed"], outputCols=[f"{col_name}_encoded"])
        stages += [indexer, encoder]
    pbar.update(1)

    # Converting 'franchise_dealer' to numeric
    df_sample = df_sample.withColumn("franchise_dealer", F.col("franchise_dealer").cast("int"))

    # Assembling features
    num_columns = [col for col in df_sample.columns if col != 'price' and col not in cat_columns]
    encoded_columns = [f"{col}_encoded" for col in cat_columns]
    feature_columns = num_columns + encoded_columns
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
    stages += [assembler]
    pbar.update(1)

    # Adding scaling to the pipeline
    scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
    stages += [scaler]

    # Creating and applying the pipeline
    pipeline = Pipeline(stages=stages)
    pipeline_model = pipeline.fit(df_sample)
    df_sample = pipeline_model.transform(df_sample)
    pbar.update(1)

    # Splitting the data
    train_df, test_df = df_sample.randomSplit([0.8, 0.2], seed=42)
    pbar.update(1)

    # Using Linear Regression
    lr = LinearRegression(featuresCol="scaled_features", labelCol="price")
    model = lr.fit(train_df)
    pbar.update(1)

# Making predictions
print("Making predictions...")
predictions = model.transform(test_df)

# Evaluating the model
print("Evaluating the model...")
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)

# Displaying results
print(f"\nTrain size: {train_df.count():,} samples")
print(f"Test size: {test_df.count():,} samples")

# Multiplying R-Squared by 100 and rounding it off
print(f"\n\nR-Squared Score (Accuracy): {round(r2 * 100)}%")

# Calculating total runtime
end_time = time.time()
total_runtime = (end_time - start_time) / 60
print(f"\nOverall runtime: {round(total_runtime)} minutes.")


Processing and Training: 100%|██████████| 6/6 [6:45:52<00:00, 4058.74s/it]


Making predictions...
Evaluating the model...

Train size: 480,538 samples
Test size: 120,299 samples


R-Squared Score (Accuracy): 78%

Overall runtime: 408 minutes.


In [ ]:
# Calculating additional metrics
mae_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mae")
mae = mae_evaluator.evaluate(predictions)

mse_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mse")
mse = mse_evaluator.evaluate(predictions)

rmse_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = rmse_evaluator.evaluate(predictions)

print("Additional Metrics:")
print(f"Mean Absolute Error: {round(mae)}")
print(f"Mean Squared Error: {round(mse)}")
print(f"Root Mean Squared Error: {round(rmse)}")

Additional Metrics:
Mean Absolute Error: 4284
Mean Squared Error: 84094306
Root Mean Squared Error: 9170


The Mean Absolute Error (`MAE`) of **\$4284**  suggests that, on average, the predicted car prices deviate from the actual prices by this amount. Given that the Mean car price is **\$29933** , this error represents about `14.3% of the mean price`, which suggests that the model performs well.



In [ ]:
'''

# Define a different path where you want to save the model
new_model_path = "/content/drive/MyDrive/Big Data Analytics - Project/models/linear_regression_model_v2"

# Save the trained model to the new path
model.save(new_model_path)

print(f"Model saved successfully at {new_model_path}")

'''


Model saved successfully at /content/drive/MyDrive/Big Data Analytics - Project/models/linear_regression_model_v2


In [ ]:
features_list = pipeline_model.stages[-2].getInputCols()  # Getting feature names from VectorAssembler

coefficients = model.coefficients  # Getting the coefficients from the linear regression model

# Sorting features by the absolute value of their coefficients in descending order
sorted_features = [feature for feature, coefficient in sorted(zip(features_list, coefficients), key=lambda x: abs(x[1]), reverse=True)]

# Printing the ranked features from highest to lowest
print("Top 10 Features Ranked by Coefficient Magnitude (Highest to Lowest):")
for rank, feature in enumerate(sorted_features[:10], 1):  # Limiting to top 10 features
    print(f"{rank}. {feature}")


Top 10 Features Ranked by Coefficient Magnitude (Highest to Lowest):
1. horsepower
2. year
3. wheelbase
4. length
5. engine_displacement
6. is_new
7. width
8. listing_color_encoded
9. major_options_count
10. make_name_encoded


In [ ]:
import pyspark

# Getting the list of all variables in the current notebook environment
for var_name in dir():
    # Checking if the variable is a Spark DataFrame
    if isinstance(eval(var_name), pyspark.sql.dataframe.DataFrame):
        # Deleting the variables
        del globals()[var_name]

# Performing garbage collection
import gc
_ = gc.collect()
print("All DataFrames have been deleted.")


spark.catalog.clearCache()
spark.stop()
print("Spark Stopped !")

All DataFrames have been deleted.
Spark Stopped !
